# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/Cities.csv")
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,colquechaca,90,BO,1.599492e+09,60,-18.67,-66.02,44.40,6.26
1,1,ushuaia,100,AR,1.599492e+09,70,-54.80,-68.30,39.20,5.82
2,2,mahebourg,75,MU,1.599492e+09,88,-20.41,57.70,73.40,11.07
3,3,liverpool,40,GB,1.599492e+09,82,53.41,-2.98,64.40,16.11
4,4,hermanus,0,ZA,1.599492e+09,70,-34.42,19.23,70.00,8.01
...,...,...,...,...,...,...,...,...,...,...
545,545,storforshei,40,NO,1.599492e+09,76,66.40,14.52,55.99,4.70
546,546,agirish,0,RU,1.599492e+09,51,61.92,63.02,51.26,4.03
547,547,moroni,40,KM,1.599492e+09,69,-11.70,43.26,78.80,8.05
548,548,taminango,75,CO,1.599492e+09,72,1.57,-77.29,66.20,3.36


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
Humidity = cities_df["Humidity"]
max_humidity = cities_df["Humidity"].max()

In [4]:
fig = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=Humidity, max_intensity=max_humidity)
fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = cities_df.loc[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 70),:]
hotel_df = hotel_df.loc[hotel_df["Wind Speed"] < 10,:]
hotel_df = hotel_df.loc[hotel_df["Cloudiness"] == 0,:]
hotel_df = hotel_df.loc[hotel_df["Humidity"] < 50,:]
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
50,50,vallenar,0,CL,1.599492e+09,28,-28.57,-70.76,70.75,4.25
148,148,mount isa,0,AU,1.599492e+09,23,-20.73,139.50,77.00,6.93
178,178,proletarsk,0,RU,1.599492e+09,31,46.70,41.72,79.88,7.45
206,206,de aar,0,ZA,1.599492e+09,20,-30.65,24.01,73.36,6.17
311,311,milyutinskaya,0,RU,1.599492e+09,24,48.63,41.67,78.84,8.90
336,336,romny,0,UA,1.599492e+09,38,50.75,33.47,79.77,5.17
384,384,chitral,0,PK,1.599492e+09,36,35.85,71.79,74.17,5.10
399,399,birjand,0,IR,1.599492e+09,12,32.87,59.22,77.00,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Lat"] = ""
hotel_df["Hotel Lng"] = ""
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
50,50,vallenar,0,CL,1.599492e+09,28,-28.57,-70.76,70.75,4.25,,,
148,148,mount isa,0,AU,1.599492e+09,23,-20.73,139.50,77.00,6.93,,,
178,178,proletarsk,0,RU,1.599492e+09,31,46.70,41.72,79.88,7.45,,,
206,206,de aar,0,ZA,1.599492e+09,20,-30.65,24.01,73.36,6.17,,,
311,311,milyutinskaya,0,RU,1.599492e+09,24,48.63,41.67,78.84,8.90,,,


In [7]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

radius = 5000

parameters = {"key": g_key,
              "radius": radius,
              "keyword": "hotel"}
failed = []

for index, row in hotel_df.iterrows():
    parameters["location"] = f"{row[6]},{row[7]}"
        
    response = requests.get(url, params=parameters).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Lat"] = response["results"][0]["geometry"]["location"]["lat"]
        hotel_df.loc[index, "Hotel Lng"] = response["results"][0]["geometry"]["location"]["lng"]
    except (KeyError, IndexError):
        print("Couldn't find a hotel...")
        failed.append(hotel_df.loc[index, "City"])

Couldn't find a hotel...


In [8]:
failed
for city in failed:
    hotel_df = hotel_df[hotel_df["City"] != city]
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
50,50,vallenar,0,CL,1.599492e+09,28,-28.57,-70.76,70.75,4.25,Hotel Puerto de Vega,-28.5783,-70.7793
148,148,mount isa,0,AU,1.599492e+09,23,-20.73,139.50,77.00,6.93,ibis Styles Mt Isa Verona,-20.7261,139.493
178,178,proletarsk,0,RU,1.599492e+09,31,46.70,41.72,79.88,7.45,vasilisa,46.7031,41.7487
206,206,de aar,0,ZA,1.599492e+09,20,-30.65,24.01,73.36,6.17,De Herberg Lodge,-30.6464,24.0056
336,336,romny,0,UA,1.599492e+09,38,50.75,33.47,79.77,5.17,ОРХІДЕЯ,50.7467,33.4829
384,384,chitral,0,PK,1.599492e+09,36,35.85,71.79,74.17,5.10,Al Mughal Hotel,35.8559,71.7875
399,399,birjand,0,IR,1.599492e+09,12,32.87,59.22,77.00,9.17,Ghaem Hotel,32.8851,59.2168


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [10]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name,Hotel Lat,Hotel Lng
50,50,vallenar,0,CL,1.599492e+09,28,-28.57,-70.76,70.75,4.25,Hotel Puerto de Vega,-28.5783,-70.7793
148,148,mount isa,0,AU,1.599492e+09,23,-20.73,139.50,77.00,6.93,ibis Styles Mt Isa Verona,-20.7261,139.493
178,178,proletarsk,0,RU,1.599492e+09,31,46.70,41.72,79.88,7.45,vasilisa,46.7031,41.7487
206,206,de aar,0,ZA,1.599492e+09,20,-30.65,24.01,73.36,6.17,De Herberg Lodge,-30.6464,24.0056
336,336,romny,0,UA,1.599492e+09,38,50.75,33.47,79.77,5.17,ОРХІДЕЯ,50.7467,33.4829
384,384,chitral,0,PK,1.599492e+09,36,35.85,71.79,74.17,5.10,Al Mughal Hotel,35.8559,71.7875
399,399,birjand,0,IR,1.599492e+09,12,32.87,59.22,77.00,9.17,Ghaem Hotel,32.8851,59.2168


In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)

fig.add_layer(markers)

# Display Map

fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
locations = cities_df[["Lat", "Lng"]]
Temp = cities_df["Max Temp"]
max_temp = cities_df["Max Temp"].max()

In [13]:
fig2 = gmaps.figure()
heatmap = gmaps.heatmap_layer(locations, weights=Temp, max_intensity=max_temp)
fig2.add_layer(heatmap)

fig2

Figure(layout=FigureLayout(height='420px'))

In [15]:
fig2.add_layer(markers)

fig2

Figure(layout=FigureLayout(height='420px'))